### Word Group In Youtube

In [153]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [154]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [155]:
# language pair
lang_folder = "Portuguese"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # native word end index
unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
word_usage_max = 100

# sentence ratio and time shift
adjust_sent_word_ratio = 30
adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [156]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [157]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [158]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [159]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [160]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [161]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [162]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,que,22182513
1,a,22151153
2,o,21751123
3,não,17756980
4,de,15384404
...,...,...
278698,diagnosian,5
278699,aconchegarmo,5
278700,pumper,5
278701,patofa,5


In [163]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [164]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,que,22182513
1,a,22151153
2,o,21751123
3,não,17756980
4,de,15384404
5,é,12104949
6,e,10217218
7,um,7748670
8,para,7166761
9,se,6364497


In [165]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [166]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [167]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

28

#### Youtube Sentence Analysis

In [168]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:09.740,00:00:11.460,olá à todos os seguidores do estevam,ThC9MSlB1gA
1,00:00:52.520,00:00:53.500,é isso mesmo,ThC9MSlB1gA
2,00:01:22.480,00:01:25.880,são temas diferentes quem tem ascendência port...,ThC9MSlB1gA
3,00:01:26.040,00:01:31.360,mas você recebeu muitas perguntas relacionadas...,ThC9MSlB1gA
4,00:01:31.380,00:01:37.080,muitas pessoas querem saber se vivendo em port...,ThC9MSlB1gA
...,...,...,...,...
1370676,00:39:59.640,00:40:04.680,coisa que vocês não gostaram e queira que eu m...,0xOfzjAWR8o
1370677,00:40:04.680,00:40:12.150,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o
1370678,00:40:12.150,00:40:18.990,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o
1370679,00:40:18.990,00:40:25.020,um beijo no coração de todos obrigada pela ate...,0xOfzjAWR8o


In [169]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [170]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,9.740,11.460,olá à todos os seguidores do estevam,ThC9MSlB1gA
1,1,52.520,53.500,é isso mesmo,ThC9MSlB1gA
2,2,82.480,85.880,são temas diferentes quem tem ascendência port...,ThC9MSlB1gA
3,3,86.040,91.360,mas você recebeu muitas perguntas relacionadas...,ThC9MSlB1gA
4,4,91.380,97.080,muitas pessoas querem saber se vivendo em port...,ThC9MSlB1gA
...,...,...,...,...,...
1370676,1370676,2399.640,2404.680,coisa que vocês não gostaram e queira que eu m...,0xOfzjAWR8o
1370677,1370677,2404.680,2412.150,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o
1370678,1370678,2412.150,2418.990,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o
1370679,1370679,2418.990,2425.020,um beijo no coração de todos obrigada pela ate...,0xOfzjAWR8o


In [171]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [172]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [173]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,9.740,11.460,olá à todos os seguidores do estevam,ThC9MSlB1gA,28.6,"estevam, à, seguidores, todos, olá","os, do"
1,1,52.520,53.500,é isso mesmo,ThC9MSlB1gA,66.6,mesmo,"é, isso"
2,2,82.480,85.880,são temas diferentes quem tem ascendência port...,ThC9MSlB1gA,0.0,"tem, quem, ascendência, diferentes, são, portu...",
3,3,86.040,91.360,mas você recebeu muitas perguntas relacionadas...,ThC9MSlB1gA,20.0,"perguntas, portugal, relacionadas, à, você, re...","em, mas"
4,4,91.380,97.080,muitas pessoas querem saber se vivendo em port...,ThC9MSlB1gA,16.7,"portugal, saber, pessoas, nacionalidade, obter...","em, se"
...,...,...,...,...,...,...,...,...
1370676,1370676,2399.640,2404.680,coisa que vocês não gostaram e queira que eu m...,0xOfzjAWR8o,29.4,"favor, gostaram, coisa, comentários, nos, deix...","eu, e, não, que, por"
1370677,1370677,2404.680,2412.150,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o,41.2,"receberem, favor, inscrevam, ajudam, fazendo, ...","e, para, se, me, no, por, isso"
1370678,1370678,2412.150,2418.990,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o,38.5,"trabalhos, vou, carinho, próximos, fazendo, vo...","eu, para, que, os, com"
1370679,1370679,2418.990,2425.020,um beijo no coração de todos obrigada pela ate...,0xOfzjAWR8o,35.7,"próximo, atenção, vídeo, pela, coração, beijo,...","e, um, no, o, de"


In [174]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,9.740,11.460,olá à todos os seguidores do estevam,ThC9MSlB1gA,28.6,"estevam, à, seguidores, todos, olá","os, do"
1,1,52.520,53.500,é isso mesmo,ThC9MSlB1gA,66.6,mesmo,"é, isso"
2,2,82.480,85.880,são temas diferentes quem tem ascendência port...,ThC9MSlB1gA,0.0,"tem, quem, ascendência, diferentes, são, portu...",NaN
3,3,86.040,91.360,mas você recebeu muitas perguntas relacionadas...,ThC9MSlB1gA,20.0,"perguntas, portugal, relacionadas, à, você, re...","em, mas"
4,4,91.380,97.080,muitas pessoas querem saber se vivendo em port...,ThC9MSlB1gA,16.7,"portugal, saber, pessoas, nacionalidade, obter...","em, se"
...,...,...,...,...,...,...,...,...
1370676,1370676,2399.640,2404.680,coisa que vocês não gostaram e queira que eu m...,0xOfzjAWR8o,29.4,"favor, gostaram, coisa, comentários, nos, deix...","eu, e, não, que, por"
1370677,1370677,2404.680,2412.150,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o,41.2,"receberem, favor, inscrevam, ajudam, fazendo, ...","e, para, se, me, no, por, isso"
1370678,1370678,2412.150,2418.990,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o,38.5,"trabalhos, vou, carinho, próximos, fazendo, vo...","eu, para, que, os, com"
1370679,1370679,2418.990,2425.020,um beijo no coração de todos obrigada pela ate...,0xOfzjAWR8o,35.7,"próximo, atenção, vídeo, pela, coração, beijo,...","e, um, no, o, de"


In [175]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,1,52.52,53.50,é isso mesmo,ThC9MSlB1gA,66.6,mesmo,"é, isso"
1,5,97.08,98.82,portanto a resposta é sim,ThC9MSlB1gA,60.0,"portanto, resposta","a, é, sim"
2,11,124.20,130.90,de,ThC9MSlB1gA,99.9,NaN,de
3,17,179.38,181.74,e eu vou dizer de forma interpolada,ThC9MSlB1gA,42.9,"vou, forma, interpolada, dizer","de, eu, e"
4,18,182.16,183.68,portanto não tem que ser seguido,ThC9MSlB1gA,33.3,"portanto, ser, tem, seguido","não, que"
...,...,...,...,...,...,...,...,...
602644,1370669,2344.04,2352.26,separados por uma correntinha e aqui nós vamos...,0xOfzjAWR8o,33.3,"finalizar, diferente, vamos, nós, separados, c...","uma, e, um, de, por"
602645,1370673,2378.06,2385.84,ficar muito longo e eu volto com vocês no próx...,0xOfzjAWR8o,35.3,"longo, tapete, finalizarmos, ficar, próximo, v...","eu, e, para, que, no, com"
602646,1370677,2404.68,2412.15,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o,41.2,"receberem, favor, inscrevam, ajudam, fazendo, ...","e, para, se, me, no, por, isso"
602647,1370678,2412.15,2418.99,os próximos trabalhos que eu vou fazendo aqui ...,0xOfzjAWR8o,38.5,"trabalhos, vou, carinho, próximos, fazendo, vo...","eu, para, que, os, com"


In [176]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
0,1,52.52,53.50,é isso mesmo,ThC9MSlB1gA,66.6,mesmo,"é, isso",é isso
1,5,97.08,98.82,portanto a resposta é sim,ThC9MSlB1gA,60.0,"portanto, resposta","a, é, sim",é sim
3,17,179.38,181.74,e eu vou dizer de forma interpolada,ThC9MSlB1gA,42.9,"vou, forma, interpolada, dizer","de, eu, e",e eu
7,23,212.60,217.84,se a pessoa no período de,ThC9MSlB1gA,66.7,"pessoa, período","a, de, no, se",se a
8,25,220.68,227.60,saliento apenas que a pessoa no momento do ped...,ThC9MSlB1gA,31.2,"portugal, saliento, pedido, apenas, tem, momen...","a, em, que, do, no",que a
...,...,...,...,...,...,...,...,...,...
602642,1370665,2296.53,2307.36,totalizarmos os três pontos altos do bloco uma...,0xOfzjAWR8o,40.0,"vamos, três, separação, correntinha, bloco, pr...","uma, o, para, do, de, os",para o
602644,1370669,2344.04,2352.26,separados por uma correntinha e aqui nós vamos...,0xOfzjAWR8o,33.3,"finalizar, diferente, vamos, nós, separados, c...","uma, e, um, de, por",por uma
602645,1370673,2378.06,2385.84,ficar muito longo e eu volto com vocês no próx...,0xOfzjAWR8o,35.3,"longo, tapete, finalizarmos, ficar, próximo, v...","eu, e, para, que, no, com",e eu
602646,1370677,2404.68,2412.15,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o,41.2,"receberem, favor, inscrevam, ajudam, fazendo, ...","e, para, se, me, no, por, isso",isso e por


In [177]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,é isso,52.520000,53.091667,é isso mesmo,ThC9MSlB1gA
1,é sim,98.402400,98.820000,portanto a resposta é sim,ThC9MSlB1gA
2,e eu,179.380000,179.717143,e eu vou dizer de forma interpolada,ThC9MSlB1gA
3,se a,212.600000,213.648000,se a pessoa no período de,ThC9MSlB1gA
4,que a,221.796129,222.316989,saliento apenas que a pessoa no momento do ped...,ThC9MSlB1gA
...,...,...,...,...,...
386489,para o,2305.594239,2306.535978,totalizarmos os três pontos altos do bloco uma...,0xOfzjAWR8o
386490,por uma,2344.880682,2345.721364,separados por uma correntinha e aqui nós vamos...,0xOfzjAWR8o
386491,e eu,2379.369505,2379.831683,ficar muito longo e eu volto com vocês no próx...,0xOfzjAWR8o
386492,isso e por,2406.936563,2407.870312,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o


In [178]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,é isso,52.220000,53.391667,é isso mesmo,ThC9MSlB1gA
1,é sim,98.102400,99.120000,portanto a resposta é sim,ThC9MSlB1gA
2,e eu,179.080000,180.017143,e eu vou dizer de forma interpolada,ThC9MSlB1gA
3,se a,212.300000,213.948000,se a pessoa no período de,ThC9MSlB1gA
4,que a,221.496129,222.616989,saliento apenas que a pessoa no momento do ped...,ThC9MSlB1gA
...,...,...,...,...,...
386489,para o,2305.294239,2306.835978,totalizarmos os três pontos altos do bloco uma...,0xOfzjAWR8o
386490,por uma,2344.580682,2346.021364,separados por uma correntinha e aqui nós vamos...,0xOfzjAWR8o
386491,e eu,2379.069505,2380.131683,ficar muito longo e eu volto com vocês no próx...,0xOfzjAWR8o
386492,isso e por,2406.636562,2408.170312,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o


In [179]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,é isso,52,53,é isso mesmo,ThC9MSlB1gA
1,é sim,98,99,portanto a resposta é sim,ThC9MSlB1gA
2,e eu,179,180,e eu vou dizer de forma interpolada,ThC9MSlB1gA
3,se a,212,214,se a pessoa no período de,ThC9MSlB1gA
4,que a,221,223,saliento apenas que a pessoa no momento do ped...,ThC9MSlB1gA
...,...,...,...,...,...
386489,para o,2305,2307,totalizarmos os três pontos altos do bloco uma...,0xOfzjAWR8o
386490,por uma,2345,2346,separados por uma correntinha e aqui nós vamos...,0xOfzjAWR8o
386491,e eu,2379,2380,ficar muito longo e eu volto com vocês no próx...,0xOfzjAWR8o
386492,isso e por,2407,2408,vocês me ajudam muito fazendo isso e por favor...,0xOfzjAWR8o


In [180]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,para que ele do que para que ele do que,974,978,recorda da ajuda que cristal deu para que ele ...,OXyb1YR6Ljc
1,para isso que que é isso como é que eu,3736,3739,estrutura para isso eu não tenho conhecimento ...,QcVZbg5BZLY
2,é isso é que está não é não é não se,2618,2620,ontem não é igual aí é isso é que está não é n...,L1WUsCNNgIY
3,para uma para uma para uma por uma,2826,2830,gosto como falar para uma para uma para uma po...,12agG_71Ftg
4,para ele o que não não não por que,558,561,está perguntando para ele o que não não não po...,RsZtDOLavso
...,...,...,...,...,...
386489,é o,113,114,o esquerdo é o coração,m147i4kjSEY
386490,é a,111,112,seu pé direito é a bomba que distrai,m147i4kjSEY
386491,e o,85,86,os exercícios para você melhorar se inscreva a...,NmQuta0MFsk
386492,e o,15,16,você entra no seu aplicativo cadastra a chave ...,eGGwrboomfw


In [181]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,para que ele do que para que ele do que,974,978,recorda da ajuda que cristal deu para que ele ...,OXyb1YR6Ljc,40
1,para isso que que é isso como é que eu,3736,3739,estrutura para isso eu não tenho conhecimento ...,QcVZbg5BZLY,60
2,é isso é que está não é não é não se,2618,2620,ontem não é igual aí é isso é que está não é n...,L1WUsCNNgIY,54
3,para uma para uma para uma por uma,2826,2830,gosto como falar para uma para uma para uma po...,12agG_71Ftg,37
4,para ele o que não não não por que,558,561,está perguntando para ele o que não não não po...,RsZtDOLavso,66
...,...,...,...,...,...,...
386489,é o,113,114,o esquerdo é o coração,m147i4kjSEY,100
386490,é a,111,112,seu pé direito é a bomba que distrai,m147i4kjSEY,100
386491,e o,85,86,os exercícios para você melhorar se inscreva a...,NmQuta0MFsk,100
386492,e o,15,16,você entra no seu aplicativo cadastra a chave ...,eGGwrboomfw,100


In [182]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,no bem como no bem que não é um,236,239,preciso da segurança para quem colocar o dinhe...,XXcoyQcv2qw
1,mas e eu com isso o que que eu,92,94,algum tempo tá bom né mas e eu com isso o que ...,vcq776nW1X8
2,não é para por isso que eu não,444,446,tá mas não é para todo mundo não é para por is...,F_bYjsjkgdE
3,se ele está bem e é o que ele,1069,1071,se ele está feliz se ele está bem e é o que el...,RsyG4KXtknE
4,como que está não é que está,212,214,das sete cidades mas olha como que está não é ...,FW0SrmJRHj0
...,...,...,...,...,...
272825,o a,455,457,botar o a alfanumérico b c e a rua d,fRyiGVmB2i0
272826,o a,569,571,então eu tenho aqui o a,fRyiGVmB2i0
272827,é a,60,62,qual é a parte da tua estrada,LBdacX2uJx8
272828,é a,736,736,é a mesma sacolinha da entrega porque a gente ...,mUKcVfz0BCk


In [183]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-183-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,no bem como no bem que não é um,236,239,preciso da segurança para quem colocar o dinhe...,XXcoyQcv2qw,38
1,mas e eu com isso o que que eu,92,94,algum tempo tá bom né mas e eu com isso o que ...,vcq776nW1X8,32
2,não é para por isso que eu não,444,446,tá mas não é para todo mundo não é para por is...,F_bYjsjkgdE,34
3,se ele está bem e é o que ele,1069,1071,se ele está feliz se ele está bem e é o que el...,RsyG4KXtknE,55
4,como que está não é que está,212,214,das sete cidades mas olha como que está não é ...,FW0SrmJRHj0,35
...,...,...,...,...,...,...
272825,o a,455,457,botar o a alfanumérico b c e a rua d,fRyiGVmB2i0,8
272826,o a,569,571,então eu tenho aqui o a,fRyiGVmB2i0,13
272827,é a,60,62,qual é a parte da tua estrada,LBdacX2uJx8,10
272828,é a,736,736,é a mesma sacolinha da entrega porque a gente ...,mUKcVfz0BCk,3


In [184]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,isso mas o que é que os,5641,5644,diz isso mas o que é que os,RIdNWADMmic
1,mas ele não é bem com a,2616,2619,mas ele não é bem com a gente,Wb5154ZxiHs
2,mas como que isso é uma,259,261,mas como que isso é uma gíria,98B7hGXu9YE
3,e os não mas e se eu os,2994,2999,né e os não mas e se eu os,TQTjOYzo9bo
4,o que que é isso se ele,1635,1638,então o que que é isso se ele,TKPATf95Ves
...,...,...,...,...,...
1308,e o,895,898,e o,jhYxmVTssmI
1309,e a,978,988,e a,Ot8GHnzfUd0
1310,e é,2311,2324,e é,40g8pAyFlmo
1311,e é,1867,1881,e é,3gUwhZHlWI0


In [185]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,e por,zUNdjtLECEY
1,e em,zOvnzmGqqzk
2,como não eu não,zNszux4O4Es
3,ele com,zH3P1Hf4x_U
4,e com,zGeyrIgS1Ug
...,...,...
433,o a,0hnzJc8PrPE
435,mas e se,0IUu992fuTU
436,mas não,0CoxySjj9c4
437,e está,0B3tzRZUl0c


In [186]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,isso mas o que é que os,5641,5644,diz isso mas o que é que os,RIdNWADMmic
1,mas ele não é bem com a,2616,2619,mas ele não é bem com a gente,Wb5154ZxiHs
2,mas como que isso é uma,259,261,mas como que isso é uma gíria,98B7hGXu9YE
3,e os não mas e se eu os,2994,2999,né e os não mas e se eu os,TQTjOYzo9bo
4,o que que é isso se ele,1635,1638,então o que que é isso se ele,TKPATf95Ves
...,...,...,...,...,...
411,a o,5808,5814,a o,cP3qQB196wA
412,o a,430,436,o a,0hnzJc8PrPE
413,o a,124,130,o a,svq7KDrLrwA
414,o a,128,134,o a,9qMisZZbKNI


In [187]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,isso mas o que é que os,5641,5644,diz isso mas o que é que os,RIdNWADMmic,https://www.youtube.com/watch?v=RIdNWADMmic&t=...
1,mas ele não é bem com a,2616,2619,mas ele não é bem com a gente,Wb5154ZxiHs,https://www.youtube.com/watch?v=Wb5154ZxiHs&t=...
2,mas como que isso é uma,259,261,mas como que isso é uma gíria,98B7hGXu9YE,https://www.youtube.com/watch?v=98B7hGXu9YE&t=...
3,e os não mas e se eu os,2994,2999,né e os não mas e se eu os,TQTjOYzo9bo,https://www.youtube.com/watch?v=TQTjOYzo9bo&t=...
4,o que que é isso se ele,1635,1638,então o que que é isso se ele,TKPATf95Ves,https://www.youtube.com/watch?v=TKPATf95Ves&t=...
...,...,...,...,...,...,...
411,a o,5808,5814,a o,cP3qQB196wA,https://www.youtube.com/watch?v=cP3qQB196wA&t=...
412,o a,430,436,o a,0hnzJc8PrPE,https://www.youtube.com/watch?v=0hnzJc8PrPE&t=...
413,o a,124,130,o a,svq7KDrLrwA,https://www.youtube.com/watch?v=svq7KDrLrwA&t=...
414,o a,128,134,o a,9qMisZZbKNI,https://www.youtube.com/watch?v=9qMisZZbKNI&t=...


In [188]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [189]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [190]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [191]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['Portuguese_Word_Group_Max_In_Youtube_Sentence_20000_Word6.csv',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_40000_Word6.csv',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_10000_Word6.csv',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_10000_Word6.xlsx',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_5000_Word6.csv',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx']

In [192]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [193]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass